# Content Based Filtering by hand


**Learning Objectives**

1. Create and compute a user feature matrix.
2. Compute where each user lies in the feature embedding space.
3. Create recommendations for new movies based on similarity measures between the user and movie feature vectors.

## Introduction

This notebook illustrates how to implement a content based filter using low level Tensorflow operations.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook

To run this lab, we need to use TensorFlow version 2.X.

In [1]:
!pip3 install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Make sure to restart your kernel to ensure this change has taken place**.

In [2]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

2.12.0


To start, we'll create our list of users, movies and features. While the users and movies represent elements in our database, for a content-based filtering method the features of the movies are likely hand-engineered and rely on domain knowledge to provide the best embedding space. Here we use the categories of Action, Sci-Fi, Comedy, Cartoon, and Drama to describe our movies (and thus our users).

In this example, we will assume our database consists of four users and six movies, listed below.

In [3]:
users = ['Ryan', 'Danielle',  'Vijay', 'Chris']
movies = [
    'Star Wars', 'The Dark Knight', 'Shrek',
    'The Incredibles', 'Bleu', 'Memento'
]
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendations = 2

### Initialize our users, movie ratings, and features

We'll need to enter the user's movie ratings and the k-hot encoded movie features matrix. Each row of the users_movies matrix represents a single user's rating (from 1 to 10) for each movie. A zero indicates that the user has not seen/rated that movie. The movies_feats matrix contains the features for each of the given movies. Each row represents one of the six movies, while the columns represent the five categories. A one indicates that a movie fits within a given genre/category.

In [4]:
# Each row represents a user's rating for the different movies.
users_movies = tf.constant([
                [4,  6,  8,  0, 0, 0],
                [0,  0, 10,  0, 8, 3],
                [0,  6,  0,  0, 3, 7],
                [10, 9,  0,  5, 0, 2]], dtype=tf.float32)

# Features of the movies one-hot encoded.
# e.g. columns could represent
# ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
movies_feats = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]], dtype=tf.float32)

### Computing the user feature matrix

We will compute the user feature matrix; that is, a matrix containing each user's embedding in the five-dimensional feature space.

**TODO 1**: Calculuate this as the matrix multiplication of the `users_movies` tensor with the `movies_feats` tensor.

In [7]:
users_feats = tf.matmul(users_movies, movies_feats)

print("Resultado de la multiplicación:")
print(users_feats)

Resultado de la multiplicación:
tf.Tensor(
[[10. 10.  8.  8.  4.]
 [ 3.  0. 10. 10. 11.]
 [13.  6.  0.  0. 10.]
 [26. 19.  5.  5. 12.]], shape=(4, 5), dtype=float32)


Next we normalize each user feature vector to sum to 1. Normalizing isn't strictly neccesary, but it makes it so that rating magnitudes will be comparable between users.

In [9]:
users_feats_nor = users_feats / tf.reduce_sum(users_feats, axis=1, keepdims=True)
users_feats_nor

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)>

#### Ranking feature relevance for each user

We can use the users_feats computed above to represent the relative importance of each movie category for each user.

In [10]:
top_users_features = tf.nn.top_k(users_feats_nor, num_feats)[1]
top_users_features

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[0, 1, 2, 3, 4],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 1, 4, 2, 3]], dtype=int32)>

In [20]:
mov = []
for i in range(num_users):
    feature_names = [features[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users[i], feature_names))

print(mov)

Ryan: ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
Danielle: ['Drama', 'Comedy', 'Cartoon', 'Action', 'Sci-Fi']
Vijay: ['Action', 'Drama', 'Sci-Fi', 'Comedy', 'Cartoon']
Chris: ['Action', 'Sci-Fi', 'Drama', 'Comedy', 'Cartoon']
[]


### Determining movie recommendations.

We'll now use the `users_feats` tensor we computed above to determine the movie ratings and recommendations for each user.

To compute the projected ratings for each movie, we compute the similarity measure between the user's feature vector and the corresponding movie feature vector.

We will use the dot product as our similarity measure. In essence, this is a weighted movie average for each user.

**TODO 2**: Implement this as a matrix multiplication. *Hint*: one of the operands will need to be transposed.

In [22]:
import numpy as np

mov_transposed = np.array(mov).T.tolist()
print(mov_transposed)

[]


In [23]:
users_ratings = tf.matmul(users_feats_nor, mov_transposed)

print("User Ratings:")
print(users_ratings)


InvalidArgumentError: ignored

The computation above finds the similarity measure between each user and each movie in our database. To focus only on the ratings for new movies, we apply a mask to the all_users_ratings matrix.

If a user has already rated a movie, we ignore that rating. This way, we only focus on ratings for previously unseen/unrated movies.

In [ ]:
users_unseen_movies = tf.equal(users_movies, tf.zeros_like(users_movies))
ignore_matrix = tf.zeros_like(tf.cast(users_movies, tf.float32))

users_ratings_new = tf.where(
    users_unseen_movies,
    users_ratings,
    ignore_matrix)

users_ratings_new

Finally, let's grab and print out the top 2 rated movies for each user.

In [ ]:
top_movies = tf.nn.top_k(users_ratings_new, num_recommendations)[1]
top_movies

In [ ]:
for i in range(num_users):
    movie_names = [movies[index] for index in top_movies[i]]
    print('{}: {}'.format(users[i], movie_names))